In [ ]:
# import libraries
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [ ]:
# declare image size for ResNet
IMAGE_SIZE = [227, 227]

train_path = '/content/drive/MyDrive/chest_xray/train'
test_path = '/content/drive/MyDrive/chest_xray/test'

In [ ]:
# import trained ResNet50V2 from keras and keep its layers untrainable
import tensorflow as tf
resnet50 = tf.keras.applications.ResNet50V2(
    input_shape=IMAGE_SIZE, weights='imagenet', include_top=False
)

for layer in resnet50.layers:
  layer.trainable = False

94674944/94668760 [==============================] - 1s 0us/step


In [ ]:
folders = glob('/content/drive/MyDrive/chest_xray/train/*')

In [ ]:
# do transfer learning by start flattenning the output
# and use it to make prediction on the training set
x = Flatten()(resnet50.output)
prediction = Dense(len(folders), activation='softmax')(x)

# create the model
model = Model(inputs=resnet50.input, outputs=prediction)

# view the structure of the model (pretty deep architecture)
model.summary()

# we'll use the cross entropy loss function and Adam optimizer
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
______________________________________________________________________________________________

In [ ]:
# prepare the data for training and testing
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(train_path, target_size=(224,224), batch_size=32, class_mode='categorical')
testing_set = test_datagen.flow_from_directory(test_path, target_size=(224,224), batch_size=32, class_mode='categorical')

Found 5218 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=testing_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_set)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
164/164 [==============================] - 2038s 12s/step - loss: 2.6407 - accuracy: 0.8716 - val_loss: 2.1817 - val_accuracy: 0.8942
Epoch 2/10
164/164 [==============================] - 111s 675ms/step - loss: 0.3648 - accuracy: 0.9660 - val_loss: 3.2378 - val_accuracy: 0.8750
Epoch 3/10
164/164 [==============================] - 109s 667ms/step - loss: 0.7456 - accuracy: 0.9545 - val_loss: 2.0972 - val_accuracy: 0.9135
Epoch 4/10
164/164 [==============================] - 109s 664ms/step - loss: 0.7031 - accuracy: 0.9559 - val_loss: 5.4519 - val_accuracy: 0.8285
Epoch 5/10
164/164 [==============================] - 109s 664ms/step - loss: 0.8368 - accuracy: 0.9585 - val_loss: 2.1523 - val_accuracy: 0.9311
Epoch 6/10
164/164 [==============================] - 109s 663ms/step - loss: 0.8838 - accuracy: 0.9669 - val_loss: 2.3313 - val_accuracy: 0.9279
Epoch 7/10
164/164 [==============================] - 108s 661ms/step - loss: 0.6488 - accuracy: 0.9713 - val_loss: 2.9923 - 